# Reproducing experimental results of LUKE on CoNLL-2003 Using Hugging Face Transformers and applying several calibration methods

The following notebook is an adapted version of one provided by the developers of LUKE, used to replicate their experiment on an existing dataset. linked here: https://github.com/studio-ousia/luke/blob/master/notebooks/huggingface_conll_2003.ipynb

In [1]:
# Currently, LUKE is only available on the master branch
!pip install seqeval git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-hkuy2940
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-hkuy2940
  Resolved https://github.com/huggingface/transformers.git to commit 80377eb018c077dba434bc8e7912bcaed3a64d09
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import unicodedata

import numpy as np
import seqeval.metrics
import spacy
import torch
from tqdm import tqdm, trange
from transformers import LukeTokenizer, LukeForEntitySpanClassification

## Loading the dataset

The test set of the CoNLL-2003 dataset (eng.testb) is placed in the current directory and loaded using `load_examples` function.

In [3]:
# Download the testb set of the CoNLL-2003 dataset
!wget https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testb

--2023-12-09 03:46:16--  https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 748096 (731K) [text/plain]
Saving to: ‘eng.testb.1’

eng.testb.1         100%[===================>] 730.56K  --.-KB/s    in 0.04s   

2023-12-09 03:46:16 (19.6 MB/s) - ‘eng.testb.1’ saved [748096/748096]



## Loading the fine-tuned model and tokenizer

We construct the model and tokenizer using the [fine-tuned model checkpoint](https://huggingface.co/studio-ousia/luke-large-finetuned-conll-2003).

In [4]:
# Load the model checkpoint
model = LukeForEntitySpanClassification.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003")
model.eval()
model.to("cuda")

# Load the tokenizer
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003")

Some weights of the model checkpoint at studio-ousia/luke-large-finetuned-conll-2003 were not used when initializing LukeForEntitySpanClassification: ['luke.embeddings.position_ids']
- This IS expected if you are initializing LukeForEntitySpanClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeForEntitySpanClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Loading the dataset

First, the documents in the dataset is loaded using the `load_documents` function. This function outputs the list of dicts which have the following three keys:
* `words`: the sequence of words
* `labels`: the sequence of gold-standard NER labels (`"MISC"`, `"PER"`, `"ORG"`, or `"LOC"`)
* `sentence_boundaries`: positions of sentence boundaries in the word sequence

The `load_examples` function creates a batch instance for each sentence in a document.
The model addresses the task by classifying all possible entity spans in a sentence into `["NIL", "MISC", "PER", "ORG", "LOC"]`, where `"NIL"` represents that the span is not an entity name (see Section 4.3 in the [original paper](https://arxiv.org/abs/2010.01057)).
Here, we create the list of all possible entity spans (character-based start and entity positions) in a sentence.
Specifically, this function returns the list of dicts with the following four keys:
* `text`: text
* `words`: the sequence of words
* `entity_spans`: the list of possible entity spans (character-based start and end positions in the `text`)
* `original_word_spans`: the list of corresponding spans of `entity_spans` in the word sequence



In [5]:
def load_documents(dataset_file):
    documents = []
    words = []
    labels = []
    sentence_boundaries = []
    with open(dataset_file) as f:
        for line in f:
            line = line.rstrip()
            if line.startswith("-DOCSTART"):
                if words:
                    documents.append(dict(
                        words=words,
                        labels=labels,
                        sentence_boundaries=sentence_boundaries
                    ))
                    words = []
                    labels = []
                    sentence_boundaries = []
                continue

            if not line:
                if not sentence_boundaries or len(words) != sentence_boundaries[-1]:
                    sentence_boundaries.append(len(words))
            else:
                items = line.split(" ")
                words.append(items[0])
                labels.append(items[-1])

    if words:
        documents.append(dict(
            words=words,
            labels=labels,
            sentence_boundaries=sentence_boundaries
        ))

    return documents


def load_examples(documents):
    examples = []
    max_token_length = 510
    max_mention_length = 30

    for document in tqdm(documents):
        words = document["words"]
        subword_lengths = [len(tokenizer.tokenize(w)) for w in words]
        total_subword_length = sum(subword_lengths)
        sentence_boundaries = document["sentence_boundaries"]

        for i in range(len(sentence_boundaries) - 1):
            sentence_start, sentence_end = sentence_boundaries[i:i+2]
            if total_subword_length <= max_token_length:
                # if the total sequence length of the document is shorter than the
                # maximum token length, we simply use all words to build the sequence
                context_start = 0
                context_end = len(words)
            else:
                # if the total sequence length is longer than the maximum length, we add
                # the surrounding words of the target sentence　to the sequence until it
                # reaches the maximum length
                context_start = sentence_start
                context_end = sentence_end
                cur_length = sum(subword_lengths[context_start:context_end])
                while True:
                    if context_start > 0:
                        if cur_length + subword_lengths[context_start - 1] <= max_token_length:
                            cur_length += subword_lengths[context_start - 1]
                            context_start -= 1
                        else:
                            break
                    if context_end < len(words):
                        if cur_length + subword_lengths[context_end] <= max_token_length:
                            cur_length += subword_lengths[context_end]
                            context_end += 1
                        else:
                            break

            text = ""
            for word in words[context_start:sentence_start]:
                if word[0] == "'" or (len(word) == 1 and is_punctuation(word)):
                    text = text.rstrip()
                text += word
                text += " "

            sentence_words = words[sentence_start:sentence_end]
            sentence_subword_lengths = subword_lengths[sentence_start:sentence_end]

            word_start_char_positions = []
            word_end_char_positions = []
            for word in sentence_words:
                if word[0] == "'" or (len(word) == 1 and is_punctuation(word)):
                    text = text.rstrip()
                word_start_char_positions.append(len(text))
                text += word
                word_end_char_positions.append(len(text))
                text += " "

            for word in words[sentence_end:context_end]:
                if word[0] == "'" or (len(word) == 1 and is_punctuation(word)):
                    text = text.rstrip()
                text += word
                text += " "
            text = text.rstrip()

            entity_spans = []
            original_word_spans = []
            for word_start in range(len(sentence_words)):
                for word_end in range(word_start, len(sentence_words)):
                    if sum(sentence_subword_lengths[word_start:word_end + 1]) <= max_mention_length:
                        entity_spans.append(
                            (word_start_char_positions[word_start], word_end_char_positions[word_end])
                        )
                        original_word_spans.append(
                            (word_start, word_end + 1)
                        )

            examples.append(dict(
                text=text,
                words=sentence_words,
                entity_spans=entity_spans,
                original_word_spans=original_word_spans,
            ))

    return examples


def is_punctuation(char):
    cp = ord(char)
    if (cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False

In [6]:
test_documents = load_documents("eng.testb")
test_examples = load_examples(test_documents)

100%|██████████| 231/231 [00:05<00:00, 42.40it/s]


## Measuring performance

We classify all possible entity spans in the test set, exclude all spans classified into the `NIL` type, and greedily select a span from the remaining spans based on the logit of its predicted entity type in descending order.
Due to  minor differences in processing, the reproduced performance is slightly lower than the performance reported in the [original paper](https://arxiv.org/abs/2010.01057) (approximately 0.1 F1).

In [7]:
batch_size = 2
all_logits = []

for batch_start_idx in trange(0, len(test_examples), batch_size):
    batch_examples = test_examples[batch_start_idx:batch_start_idx + batch_size]
    texts = [example["text"] for example in batch_examples]
    entity_spans = [example["entity_spans"] for example in batch_examples]

    inputs = tokenizer(texts, entity_spans=entity_spans, return_tensors="pt", padding=True)
    inputs = inputs.to("cuda")
    with torch.no_grad():
        outputs = model(**inputs)
    all_logits.extend(outputs.logits.tolist())

100%|██████████| 1727/1727 [09:13<00:00,  3.12it/s]


In [8]:
final_labels = [label for document in test_documents for label in document["labels"]]

calibration_logits = []

uncalibrated_predictions = []

logits_as_one = np.array(all_logits[0])

golds_as_one = np.array([])

hist_bins = []

for example_index, example in enumerate(test_examples):
    logits = all_logits[example_index]
    max_logits = np.max(logits, axis=1)
    max_indices = np.argmax(logits, axis=1)
    original_spans = example["original_word_spans"]
    predictions = []

    for logit, index, span, logs in zip(max_logits, max_indices, original_spans, logits):
        if index != 0:  # the span is not NIL
            predictions.append((logit, span, model.config.id2label[index], logs))

    # construct an IOB2 label sequence
    predicted_sequence = ["O"] * len(example["words"])
    predicted_logits = [[1, 0, 0, 0, 0]] * len(example["words"])

    for _, span, label, logs in sorted(predictions, key=lambda o: o[0], reverse=True):
        if all([o == "O" for o in predicted_sequence[span[0] : span[1]]]):
            predicted_sequence[span[0]] = "B-" + label
            predicted_logits[span[0]] = logs
            if span[1] - span[0] > 1:
                predicted_sequence[span[0] + 1 : span[1]] = ["I-" + label] * (span[1] - span[0] - 1)
                predicted_logits[span[0] + 1: span[1]] = [logs] * (span[1] - span[0] - 1)

    uncalibrated_predictions += predicted_sequence
    calibration_logits += predicted_logits

translation = {"O": "NIL","I-LOC": "LOC","I-ORG": "ORG","I-PER": "PER","I-MISC": "MISC","B-LOC": "LOC","B-ORG": "ORG","B-PER": "PER","B-MISC": "MISC"}

gold_labels = [model.config.label2id[translation[label]] for label in final_labels]

from scipy.special import softmax

probs_for_binning = softmax(calibration_logits, axis = 1)


In [9]:
class ExpectedCalibrationError():

    def __init__(self):
        self.conf = []  # Initiate confidence list


    def _get_conf_acc(self, lower_bound, upper_bound, probs, true):
        # Filter labels within probability range

        arg_max = np.argmax(probs, axis = 1)
        max = np.max(probs, axis = 1)

        acc_filtered = [x[0] == x[2] for x in zip(true, max, arg_max) if x[1] > lower_bound and x[1] <= upper_bound]

        conf_filtered = [x[1] for x in zip(true, max, arg_max) if x[1] > lower_bound and x[1] <= upper_bound]
        nr_elems = len(acc_filtered)

        if nr_elems < 1:
            return 0, 0, 0
        else:
            acc = np.sum(acc_filtered)/nr_elems  # Sums positive classes
            conf = np.sum(conf_filtered)/nr_elems # Sums maximum probabilities
            return conf, acc, nr_elems


    def ECE(self, probs, true):

        self.upper_bounds = np.histogram_bin_edges(probs, bins = 100, range = (0, 1))[1:].tolist() + [1]
        conf = []
        acc = []
        bin_sizes = []

        # Go through intervals and add confidence to list
        for i, conf_thresh in enumerate(self.upper_bounds):
            temp_conf, temp_acc, bin_size = self._get_conf_acc(self.upper_bounds[i - 1] if i > 0 else 0, conf_thresh, probs = probs, true = true)
            conf.append(temp_conf)
            acc.append(temp_acc)
            bin_sizes.append(bin_size)

        len = np.sum(bin_sizes)

        ECE = np.sum([size * abs(accuracy - confidence) for accuracy, confidence, size in zip(acc, conf, bin_sizes)])/len

        return ECE

In [10]:
print("ECE:")
print(ExpectedCalibrationError().ECE(probs_for_binning, gold_labels))

print(seqeval.metrics.classification_report([final_labels], [uncalibrated_predictions], digits=4))

ECE:
0.4957551166667858
              precision    recall  f1-score   support

         LOC     0.9558    0.9478    0.9518      1666
        MISC     0.8553    0.8688    0.8620       701
         ORG     0.9287    0.9496    0.9391      1647
         PER     0.9683    0.9719    0.9701      1602

   micro avg     0.9386    0.9453    0.9420      5616
   macro avg     0.9270    0.9345    0.9307      5616
weighted avg     0.9389    0.9453    0.9421      5616



## Measuring Performance With Calibration

We will use several different calibration methods in post-processing as described in the paper: https://arxiv.org/pdf/1706.04599.pdf

1. Histogram Binning

In [11]:
class HistogramBinning():

    def __init__(self):
        self.conf = []


    def confidence(self, lower_bound, upper_bound, probs, true):
        filtered = [x[0] for x in zip(true, probs) if x[1] > lower_bound and x[1] <= upper_bound]
        nr_elems = len(filtered)

        if nr_elems < 1:
            return 0
        else:
            conf = np.sum(filtered)/nr_elems
            return conf


    def train(self, probs, true):

        self.upper_bounds = np.histogram_bin_edges(probs, bins = 50, range = (0, 0))[1:].tolist() + [1]

        conf = []

        # Got through intervals and add confidence to list
        for i, conf_thresh in enumerate(self.upper_bounds):
            temp_conf = self.confidence(self.upper_bounds[i - 1] if i > 0 else 0, conf_thresh, probs = probs, true = true)
            conf.append(temp_conf)

        self.conf = conf

    def predict(self, probs):
        new_prob = list(probs)
        for i, prob in enumerate(probs):
            idx = np.searchsorted(self.upper_bounds, prob)
            item = self.conf[idx]
            new_prob[i] = item
        return new_prob

In [12]:
final_labels = [label for document in test_documents for label in document["labels"]]

final_predictions = []

from scipy.special import softmax

probs_for_binning = softmax(calibration_logits, axis = 1)

In [13]:
final_predictions = []

hist_bins = []

logit_bits = []

translation = {"O": "NIL","I-LOC": "LOC","I-ORG": "ORG","I-PER": "PER","I-MISC": "MISC","B-LOC": "LOC","B-ORG": "ORG","B-PER": "PER","B-MISC": "MISC"}

for i in range(len(calibration_logits[0])):
  golds_as_one = [model.config.label2id[translation[label]] == i for label in final_labels]
  bin = HistogramBinning()
  bin.train(probs_for_binning[:20000, i], golds_as_one[:20000])
  hist_bins.append(bin)

for i in range(len(hist_bins)):
  logit_bits.append(hist_bins[i].predict(probs_for_binning[:, i]))

logits = np.stack(logit_bits, axis = 1)

calibrated_predictions = []

indices = np.argmax(logits, axis = 1)

for index in indices:
  label = model.config.id2label[index]
  label = "O" if label == "NIL" else "I-" + label
  calibrated_predictions.append(label)

print("ECE:")
print(ExpectedCalibrationError().ECE(softmax(logits, axis = 1), gold_labels))

print(seqeval.metrics.classification_report([final_labels], [calibrated_predictions], digits=4))

ECE:
0.5865472775873256
              precision    recall  f1-score   support

         LOC     0.9569    0.9472    0.9520      1666
        MISC     0.8621    0.8559    0.8590       701
         ORG     0.9323    0.9454    0.9388      1647
         PER     0.9793    0.9769    0.9781      1602

   micro avg     0.9442    0.9437    0.9440      5616
   macro avg     0.9327    0.9313    0.9320      5616
weighted avg     0.9443    0.9437    0.9440      5616



2. Isotonic Regression

In [14]:
from sklearn.isotonic import IsotonicRegression

In [15]:
final_predictions = []

regs = []

logit_bits = []

translation = {"O": "NIL","I-LOC": "LOC","I-ORG": "ORG","I-PER": "PER","I-MISC": "MISC","B-LOC": "LOC","B-ORG": "ORG","B-PER": "PER","B-MISC": "MISC"}

for i in range(len(calibration_logits[0])):
  golds_as_one = [model.config.label2id[translation[label]] == i for label in final_labels]
  reg = IsotonicRegression()
  reg.fit(probs_for_binning[20000:, i], golds_as_one[20000:])
  regs.append(reg)

for i in range(len(regs)):
  logit_bits.append(regs[i].predict(probs_for_binning[:, i]))

logits = np.stack(logit_bits, axis = 1)

calibrated_predictions = []

indices = np.argmax(logits, axis = 1)

calibrated_predictions = []

for index in indices:
  label = model.config.id2label[index]
  label = "O" if label == "NIL" else "I-" + label
  calibrated_predictions.append(label)

print("ECE:")
print(ExpectedCalibrationError().ECE(softmax(logits, axis = 1), gold_labels))

print(seqeval.metrics.classification_report([final_labels], [calibrated_predictions], digits=4))

ECE:
0.5873015979076123
              precision    recall  f1-score   support

         LOC     0.9575    0.9460    0.9517      1666
        MISC     0.8620    0.8645    0.8632       701
         ORG     0.9329    0.9460    0.9394      1647
         PER     0.9824    0.9763    0.9793      1602

   micro avg     0.9453    0.9444    0.9449      5616
   macro avg     0.9337    0.9332    0.9334      5616
weighted avg     0.9455    0.9444    0.9449      5616



3. Matrix Scaling

In [16]:
from sklearn.linear_model import LogisticRegression

In [17]:
final_predictions = []

regs = []

logit_bits = []

translation = {"O": "NIL","I-LOC": "LOC","I-ORG": "ORG","I-PER": "PER","I-MISC": "MISC","B-LOC": "LOC","B-ORG": "ORG","B-PER": "PER","B-MISC": "MISC"}

log_model = LogisticRegression(multi_class = 'ovr')

gold_labels = [model.config.label2id[translation[label]] for label in final_labels]

log_model = log_model.fit(calibration_logits, gold_labels)

log_probs = log_model.predict_log_proba(calibration_logits)

indices = np.argmax(log_probs, axis = 1)

calibrated_predictions = []

for index in indices:
  label = model.config.id2label[index]
  label = "O" if label == "NIL" else "I-" + label
  calibrated_predictions.append(label)

print("ECE:")
print(ExpectedCalibrationError().ECE(softmax(log_probs, axis = 1), gold_labels))

print(seqeval.metrics.classification_report([final_labels], [calibrated_predictions], digits=4))

ECE:
0.0055017690648463905
              precision    recall  f1-score   support

         LOC     0.9605    0.9334    0.9467      1666
        MISC     0.8957    0.7960    0.8429       701
         ORG     0.9344    0.9423    0.9383      1647
         PER     0.9798    0.9707    0.9752      1602

   micro avg     0.9508    0.9295    0.9400      5616
   macro avg     0.9426    0.9106    0.9258      5616
weighted avg     0.9503    0.9295    0.9394      5616



4. Temperature Scaling

In [18]:
final_predictions = []

regs = []

logit_bits = []

translation = {"O": "NIL","I-LOC": "LOC","I-ORG": "ORG","I-PER": "PER","I-MISC": "MISC","B-LOC": "LOC","B-ORG": "ORG","B-PER": "PER","B-MISC": "MISC"}

gold_labels = [model.config.label2id[translation[label]] for label in final_labels]

temp_logits = np.copy(calibration_logits)

temperature = .1

temp_logits = temp_logits / temperature

indices = np.argmax(temp_logits, axis = 1)

calibrated_predictions = []

for index in indices:
  label = model.config.id2label[index]
  label = "O" if label == "NIL" else "I-" + label
  calibrated_predictions.append(label)

print("ECE:")
print(ExpectedCalibrationError().ECE(softmax(temp_logits, axis = 1), gold_labels))

print(seqeval.metrics.classification_report([final_labels], [calibrated_predictions], digits=4))

ECE:
0.012199425990354187
              precision    recall  f1-score   support

         LOC     0.9552    0.9466    0.9509      1666
        MISC     0.8623    0.8573    0.8598       701
         ORG     0.9296    0.9460    0.9377      1647
         PER     0.9781    0.9769    0.9775      1602

   micro avg     0.9426    0.9439    0.9432      5616
   macro avg     0.9313    0.9317    0.9315      5616
weighted avg     0.9426    0.9439    0.9432      5616

